In [1]:
# Sheet properties

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [3]:
# Imports

In [4]:
import sys
sys.path.append('/home/rcendre/classification')
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  
from numpy import array, ones, maximum
from sklearn.preprocessing import LabelEncoder
from toolbox.models.builtin import Applications
from toolbox.classification.common import IO, Tools, Folds
from toolbox.classification.parameters import Settings, Dermatology
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import Views, ViewsTools
import cv2
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Parameters

In [6]:
validation = 4
settings = Settings.get_default_dermatology()

In [7]:
# Inputs

In [8]:
inputs = Dermatology.images(modality='Microscopy', data_type='Full', use_unknown=False)

In [9]:
# Transform groups
group_encoder = LabelEncoder().fit(array(inputs['ID_Patient'].tolist()))
Tools.transform(inputs, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
Tools.transform(inputs, {'datum': 'Label'}, label_encoder, 'LabelEncode')
# Make folds
Folds.build_group_folds(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode', 'group': 'GroupEncode'}, validation)

In [10]:
# Additionnal

In [11]:
additionnal = {'batch_size': 8,
            'epochs' : 15,
            'shuffle': True,
            'rotation_range': 45,
            'width_shift_range': 0.1,
            'height_shift_range': 0.1,
            'horizontal_flip': True,
            'vertical_flip': True,
            'fill_mode': 'wrap'}
model = Applications.get_fine_tuning(3, 0, -1, architecture='ResNet', pooling='max', activation='softmax', additional=additionnal)

In [12]:
# Models

In [13]:
model = Tools.fit(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode'}, model)
model.save('FineVisualisation')

Found 5431 images belonging to 3 classes.
Pre-training...
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 679 steps
Epoch 1/15
679/679 [==============================] - 1146s 2s/step - loss: 6.1036 - accuracy: 0.5404
Epoch 2/15
679/679 [==============================] - 1146s 2s/step - loss: 5.4405 - accuracy: 0.6080
Epoch 3/15
679/679 [==============================] - 1150s 2s/step - loss: 4.7026 - accuracy: 0.6369
Epoch 4/15
679/679 [==============================] - 1157s 2s/step - loss: 4.1553 - accuracy: 0.6586
Epoch 5/15
679/679 [==============================] - 1145s 2s/step - loss: 4.2468 - accuracy: 0.6684
Epoch 6/15
679/679 [==============================] - 1142s 2s/step - loss: 3.8542 - accuracy: 0.6844
Epoch 7/15
679/679 [==============================] - 1145s 2s/step - loss: 4.4743 - accuracy: 0.6702
Epoch 8/15
679/679 [==============================] - 1144s 2s/step - loss: 4.1005 - accuracy: 0.6927
Epoch

In [14]:
# Visualisation

In [ ]:
model.load('FineVisualisation')

from toolbox.views.images import GradCAM

for IMAGE_INDEX in range(0,1000):
    image_class = inputs['LabelEncode'][IMAGE_INDEX]
    image_path = inputs['Datum'][IMAGE_INDEX]
    image = tf.keras.preprocessing.image.load_img(image_path)
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image_size = image.shape

    # initialize our gradient class activation map and build the heatmap
    cam = GradCAM(model.model, image_class)
    heatmap = cam.compute_heatmap(image)
    cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

    output_image = cv2.addWeighted(cv2.cvtColor(image[0].astype('uint8'), cv2.COLOR_RGB2BGR), 1, cam, 0.25, 0)

    cv2.imwrite(f'FineVis/{IMAGE_INDEX}_{image_class}.png', output_image)

Using TensorFlow backend.
